In [7]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

import scipy.stats as stats
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import *

# Z-критерий для двух долей

In [4]:
data = pd.read_csv('banner_click_stat.txt', header=None, sep='\t')
data.columns = ['BannerA', 'BannerB']

,BannerA,BannerB
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,0
997,0,0
998,0,0


In [5]:
data.head()

,BannerA,BannerB
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


In [6]:
data.describe()

,BannerA,BannerB
count,1000.000000,1000.000000
mean,0.037000,0.053000
std,0.188856,0.224146
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


## Интервальные оценки для долей

In [14]:
def quantile(input_data, alpha=0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    p = sum(input_data) / len(input_data)
    add_sub = z * np.sqrt(p * (1 - p) / len(input_data))
    return p - add_sub, p + add_sub

print("quantile interval A: [{0[0]:.4f},{0[1]:.4f}]".format(quantile(data.BannerA)))
print("quantile interval B: [{0[0]:.4f},{0[1]:.4f}]".format(quantile(data.BannerB)))

quantile interval A: [0.0253,0.0487]
quantile interval B: [0.0391,0.0669]


In [13]:
proportion_confint(sum(data.BannerA), data.shape[0], alpha=.05, method="normal")

(0.02530064022092865, 0.04869935977907135)

## Z-критерий для разности долей для независимых выборок

In [18]:
def proportions_diff_confint_ind(data1, data2, alpha=0.05):
    p1 = sum(data1) / len(data1)
    p2 = sum(data2) / len(data2)
    z = stats.norm.ppf(1 - alpha/2.)
    add_sub = z * (np.sqrt((p1 * (1 - p1) / len(data1)) + (p2 * (1 - p2) / len(data2))))
    P = (p1 * len(data1) + p2 * len(data2)) / (len(data1) + len(data2))
    Z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * ((1 / len(data1)) + (1 / len(data2))))
    return p1 - p2 - add_sub, p1 - p2 + add_sub, Z_stat


def proportions_diff_z_test(z_stat, alternative="two-sided"):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError('alternative error')
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    if alternative == 'less':
        return stats.norm.cdf(z_stat)
    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [24]:
lower, upper, Z_stat = proportions_diff_confint_ind(data.BannerA, data.BannerB, alpha=.05)

In [29]:
print('quantile: [{0:.4f},{1:.4f}]'.format(lower, upper))
print(f'p-value: {proportions_diff_z_test(Z_stat):.4f}')
print(f'p-value less: {proportions_diff_z_test(Z_stat, "less"):.4f}')

quantile: [-0.0342,0.0022]
p-value: 0.0844
p-value less: 0.0422


#   Со связными выборками все очень похоже, но другая диагональ (функции как-то не хочется писать)